In [1]:
import keras
import numpy as np
import tensorflow as tf
import datetime
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import StandardScaler

2024-04-23 18:09:16.192695: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 18:09:16.233611: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 18:09:16.233639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 18:09:16.234606: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 18:09:16.241449: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 18:09:16.242256: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
X, y = fetch_california_housing(return_X_y=True)

In [3]:
X = StandardScaler().fit_transform(X)
X.shape

(20640, 8)

In [4]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, random_state=42, train_size=0.75)

In [5]:
from functools import partial
def Model():
    Dense =  partial(
        keras.layers.Dense,
        activation = keras.activations.selu,
        kernel_initializer = keras.initializers.lecun_normal     
    )
    model = keras.models.Sequential(
        layers=[
            Dense(16),
            Dense(8),
            Dense(4),
            Dense(2),
            keras.layers.Dense(1)
        ]
    )
    return model

## customizando modelos

In [6]:
def huder_fn(ytrue, ypred):
    error = ytrue - ypred
    is_small_error = tf.abs(error)<1
    squared_loss   = tf.square(error)/2
    linear_loss    = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [7]:
model = Model()
path  ='logs/fit/'+datetime.datetime.now().strftime("%Y_%m_%d - %H_%M_%S")
tensorboard = keras.callbacks.TensorBoard(log_dir=path, histogram_freq=1)
model.compile(
    loss=huder_fn,
    optimizer=keras.optimizers.RMSprop(),
)
model.fit(xtrain, ytrain, batch_size=32, epochs=20, callbacks=[tensorboard])

Epoch 1/20
484/484 [==============================] - 4s 4ms/step - loss: 0.6533
Epoch 2/20
484/484 [==============================] - 2s 4ms/step - loss: 0.2549
Epoch 3/20
484/484 [==============================] - 2s 4ms/step - loss: 0.1951
Epoch 4/20
484/484 [==============================] - 2s 3ms/step - loss: 0.1733
Epoch 5/20
484/484 [==============================] - 2s 3ms/step - loss: 0.1653
Epoch 6/20
484/484 [==============================] - 2s 3ms/step - loss: 0.1607
Epoch 7/20
484/484 [==============================] - 2s 3ms/step - loss: 0.1583
Epoch 8/20
484/484 [==============================] - 2s 3ms/step - loss: 0.1564
Epoch 9/20
484/484 [==============================] - 2s 4ms/step - loss: 0.1542
Epoch 10/20
484/484 [==============================] - 2s 4ms/step - loss: 0.1531
Epoch 11/20
484/484 [==============================] - 1s 3ms/step - loss: 0.1515
Epoch 12/20
484/484 [==============================] - 1s 3ms/step - loss: 0.1511
Epoch 13/20
484/484 [====

In [8]:
predict = model.predict(xtest)

162/162 [==============================] - 1s 2ms/step


In [9]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
print(tf.math.reduce_std(y))

result = f'''
mean_absolute_error = {mean_absolute_error(ytest ,predict)}
mean_squared_error  = {mean_squared_error(ytest, predict)}
r2_score            = {r2_score(ytest, predict)}

'''
print(result)

tf.Tensor(1.153928204041225, shape=(), dtype=float64)

mean_absolute_error = 0.3927523056996257
mean_squared_error  = 0.33517803535921464
r2_score            = 0.746694793462471




In [10]:
def create_huber(threshold=1.0):
    def huder_fn(ytrue, ypred):
        error = ytrue - ypred
        is_small_error = tf.abs(error) < threshold
        squared_loss   = tf.square(error) / 2
        linear_loss    = threshold * tf.abs(error) - threshold**2 /2
        return tf.where( is_small_error, squared_loss, linear_loss)
    return  huder_fn

In [11]:
model = Model()

model.compile(
    loss=create_huber(2.0),
    optimizer=keras.optimizers.RMSprop(),
)
model.fit(xtrain, ytrain, batch_size=32, epochs=20, callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=3)])

Epoch 1/20
484/484 [==============================] - 3s 3ms/step - loss: 0.4061
Epoch 2/20
484/484 [==============================] - 1s 3ms/step - loss: 0.2227
Epoch 3/20
484/484 [==============================] - 1s 2ms/step - loss: 0.2099
Epoch 4/20
484/484 [==============================] - 1s 2ms/step - loss: 0.2025
Epoch 5/20
484/484 [==============================] - 1s 2ms/step - loss: 0.1972
Epoch 6/20
484/484 [==============================] - 1s 2ms/step - loss: 0.1930
Epoch 7/20
484/484 [==============================] - 1s 2ms/step - loss: 0.1888
Epoch 8/20
484/484 [==============================] - 1s 2ms/step - loss: 0.1838
Epoch 9/20
484/484 [==============================] - 1s 2ms/step - loss: 0.1792
Epoch 10/20
484/484 [==============================] - 1s 2ms/step - loss: 0.1752
Epoch 11/20
484/484 [==============================] - 1s 2ms/step - loss: 0.1733
Epoch 12/20
484/484 [==============================] - 1s 2ms/step - loss: 0.1702
Epoch 13/20
484/484 [====

In [12]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
print(tf.math.reduce_std(y))

result = f'''
mean_absolute_error = {mean_absolute_error(ytest ,predict)}
mean_squared_error  = {mean_squared_error(ytest, predict)}
r2_score            = {r2_score(ytest, predict)}

'''
print(result)

tf.Tensor(1.153928204041225, shape=(), dtype=float64)

mean_absolute_error = 0.3927523056996257
mean_squared_error  = 0.33517803535921464
r2_score            = 0.746694793462471




construindo por meio de class

In [13]:
class HuderLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, ytrue, ypred):
        error = ytrue - ypred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss   = tf.square(error) / 2
        linear_loss    = self.threshold * tf.abs(error) - self.threshold**2 /2
        return tf.where( is_small_error, squared_loss, linear_loss)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}
        

In [16]:
model = Model()

model.compile(
    loss=HuderLoss(2.0),
    optimizer=keras.optimizers.RMSprop(),
)
model.fit(xtrain, ytrain, batch_size=32, epochs=20)

Epoch 1/20
484/484 [==============================] - 1s 1ms/step - loss: 0.5805 - huder_fn: 0.5805
Epoch 2/20
484/484 [==============================] - 1s 1ms/step - loss: 0.2514 - huder_fn: 0.2514
Epoch 3/20
484/484 [==============================] - 1s 1ms/step - loss: 0.2149 - huder_fn: 0.2149
Epoch 4/20
484/484 [==============================] - 1s 1ms/step - loss: 0.2007 - huder_fn: 0.2007
Epoch 5/20
484/484 [==============================] - 1s 1ms/step - loss: 0.1935 - huder_fn: 0.1935
Epoch 6/20
484/484 [==============================] - 1s 1ms/step - loss: 0.1886 - huder_fn: 0.1886
Epoch 7/20
484/484 [==============================] - 1s 1ms/step - loss: 0.1849 - huder_fn: 0.1849
Epoch 8/20
484/484 [==============================] - 1s 1ms/step - loss: 0.1823 - huder_fn: 0.1823
Epoch 9/20
484/484 [==============================] - 1s 1ms/step - loss: 0.1793 - huder_fn: 0.1793
Epoch 10/20
484/484 [==============================] - 1s 1ms/step - loss: 0.1778 - huder_fn: 0.1778